In [49]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cProfile

In [50]:
def lagrange_descent(object_func, learning_rate, iterations, t_max):

    search_radius = learning_rate # length of the line from initial point

    # initialize plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # initialize lists to store min_vals
    x_points = []
    y_points = []
    z_points = []

    # plot the 3D surface for L - just for visualization purposes
    x_surface = np.linspace(-10, 10, 100)  
    y_surface = np.linspace(-10, 10, 100)
    x_surface, y_surface = np.meshgrid(x_surface, y_surface)
    # z_surface = 3*x_surface**2 + 0.7*y_surface**2  - L appears here
    z_surface = x_surface**4 - 5*x_surface**3
    ax.plot_surface(x_surface, y_surface, z_surface, alpha=0.7, rstride=10, cstride=10, linewidth=0.5)

    for i in range(iterations):

        if i == 0:
            x0, y0 = np.random.uniform(-15, 15, 2)
            # x0 , y0 = -8, 4
            if x0 > 0:
                # theta = np.pi + np.arctan((1.4*y0)/(6*x0)) if x0 != 0 else np.random.uniform(0, 2*np.pi)
                theta = np.pi + np.arctan((y0)/(3*x0**3 - 15*x0**2)) if x0 != 0 else np.random.uniform(0, 2*np.pi)

            else:
                # theta = np.arctan((1.4*y0)/(6*x0)) if x0 != 0 else np.random.uniform(0, 2*np.pi)
                theta = np.arctan((y0)/(3*x0**3 - 15*x0**2)) if x0 != 0 else np.random.uniform(0, 2*np.pi)

            x1 = x0 + search_radius*np.cos(theta)
            y1 = y0 + search_radius*np.sin(theta)
        else: 
            x0, y0 = min_vals[0,0], min_vals[0,1]
            if x0 >= 0:
                # theta = np.pi + np.arctan((1.4*y0)/(6*x0)) if x0 != 0 else np.random.uniform(0, 2*np.pi)
                theta = np.pi + np.arctan((y0)/(3*x0**3 - 15*x0**2)) if x0 != 0 else np.random.uniform(0, 2*np.pi)

            else:
                # theta = np.arctan((1.4*y0)/(6*x0)) if x0 != 0 else np.random.uniform(0, 2*np.pi)
                theta = np.arctan((y0)/(3*x0**3 - 15*x0**2)) if x0 != 0 else np.random.uniform(0, 2*np.pi)

            x1 = x0 + search_radius*np.cos(theta)
            y1 = y0 + search_radius*np.sin(theta)

        # calculate the coefficients of the straight line path y = mx + c
        m = (y1 - y0) / (x1 - x0)
        c = y0 - m * x0

        # generate points along the path
        t_values = np.linspace(x0, x1, t_max) # parametrization of line

        # first line: create array of points along line path
        line_vals = np.empty([t_max, 3])     # initialize empty array
        line_vals[:,0] = t_values            # populate array x-values
        line_vals[:,1] = m*t_values + c      # populate array y-values

        for j in range(t_max):

            # line_vals[j, 2] = (line_vals[j,0])**2 + (line_vals[j,1])**2 # populate array z-values
            line_vals[j, 2] = (line_vals[j,0])**4 - 5*(line_vals[j,1])**3 # populate array z-values - L appears here

        # return the coordinates to the point on the line with minimal z value
        min_vals = line_vals[line_vals[:,2] == np.min(line_vals[:,2])]

        # re-initialize points of the starting line
        x0, y0 = min_vals[0,0], min_vals[0,1]

        # display results of each iteration
        print(f"Iteration {i}: min_vals = {min_vals[0]}, theta = {theta}")

        # collect points for plotting
        if i > 0:  
            x_points.append(min_vals[0, 0])
            y_points.append(min_vals[0, 1])
            z_points.append(min_vals[0, 2])


    # plot the points
    ax.plot(x_points, y_points, z_points, marker='o', color='r', label='Min path')   

    # set labels/legend for plot
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_zlabel('Z Axis')
    plt.title('Minimum Values per Iteration')
    ax.legend()
    plt.show()

In [51]:
t = sp.symbols('t')

# define functions
x = sp.Function('x')(t)
y = sp.Function('y')(t)
z = sp.Function('z')(t)

# define objective function
L = 3*x**2 + 0.7*y**2

In [52]:
%matplotlib qt
# call optimizer on objective function
lagrange_descent(L, 15, 25, 100)

Iteration 0: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.03728937953896934
Iteration 1: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 2: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 3: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 4: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 5: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 6: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 7: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 8: min_vals = [-2.12577538e+00  8.24458642e+00 -2.78163418e+03], theta = -0.08513920912523319
Iteration 9: min_vals = [-2.12577538e+00  8.24458642e+00 -2.7816

In [53]:
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# x = np.linspace(-3,6, 100)
# y = np.linspace(-3,6, 100)
# # z = 6*x + 1.4*y
# # theta = np.arctan((1.4*u)/(6*v))
# u, v = np.meshgrid(x,y)
# z = u**4 -5*u**3
# # plt.quiver(x,y,u,v)
# ax.plot_surface(u, v, z)

In [54]:
# cProfile.run('lagrange_descent(L, 8, 25, 1000)')